# Occupancy detection

### A multivariate time series example

In this tutorial, you will learn how to apply getML to multivariate time series. It also demonstrates how to use getML's [high-level interface for hyperparameter tuning](https://docs.getml.com/latest/user_guide/hyperopt/hyperopt.html#tuning-routines).

Summary:

- Prediction type: __Binary classification__
- Domain: __Energy__
- Prediction target: __Room occupancy__
- Source data: __1 table, 32k rows__
- Population size: __32k__

_Author: Dr. Johannes King_

# Background

Our use case is a public domain data set for predicting room occupancy from sensor data. The results achieved using getML outperform all published results on this data set. Note that this is not only a neat use case for machine learning algorithms, but a real-world application with tangible consequences: If room occupancy is known with sufficient certainty, it can be applied to the control systems of a building. Such as system can reduce the energy consumption by [up to 50 %](https://ieeexplore.ieee.org/document/7566062). Fot further details about the data set refer to [link to the occupancy notebook](#).

### Comparison of propisitionalization approaches

Approaches to propositionalization:
- getML (FastProp)
- featurettools
- tsfresh

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set-up your session:

In [1]:
import os
from urllib import request

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("seaborn")
%matplotlib inline

import getml

from utils import FTTimeSeriesBuilder, TSFreshBuilder

print(f"getML API version: {getml.__version__}\n")

getml.engine.set_project("occupancy")

getML API version: 0.16.0


Connected to project 'occupancy'


## 1. Loading data


The data set can be downloaded directly from GitHub. It is conveniently separated into a train, a validation and a testing set. This allows us to directly benchmark our results against the results of the original paper later.

In [2]:
data_test, data_train, data_validate = getml.datasets.load_occupancy(
    roles=True
).values()

The train set looks like this:

In [3]:
data_train

Name,date,Occupancy,Temperature,Humidity,Light,CO2,HumidityRatio
Role,time_stamp,target,numerical,numerical,numerical,numerical,numerical
Units,"time stamp, comparison only",,,,,,
0,2015-02-04 17:51:00,1,23.18,27.272,426,721.25,0.004793
1,2015-02-04 17:51:59,1,23.15,27.2675,429.5,714,0.004783
2,2015-02-04 17:53:00,1,23.15,27.245,426,713.5,0.004779
3,2015-02-04 17:54:00,1,23.15,27.2,426,708.25,0.004772
4,2015-02-04 17:55:00,1,23.1,27.2,426,704.5,0.004757
,...,...,...,...,...,...,...
8138,2015-02-10 09:29:00,1,21.05,36.0975,433,787.25,0.005579
8139,2015-02-10 09:29:59,1,21.05,35.995,433,789.5,0.005563


## 2. Predictive modeling

We loaded the data, defined the roles, units and the abstract data model. Next, we create a getML pipeline for relational learning.

### 2.1 Propositionalization with getML's FastProp

In [4]:
population = getml.data.Placeholder("population")
peripheral = getml.data.Placeholder("peripheral")

population.join(
    peripheral,
    time_stamp="date",
    # We want our time series features to only use
    # data from the last 15 minutes
    memory=getml.data.time.minutes(15),
    # Our forecast horizon is 0.
    # We do not predict the future, instead we infer
    # the present state from current and past sensor data.
    horizon=0.0,
    # We do not allow the time series features
    # to use target values from the past.
    allow_lagged_targets=False,
)

population

feature_learner = getml.feature_learning.FastPropModel(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    aggregation=getml.feature_learning.FastPropModel.agg_sets.All,
)

Next, we create the pipeline. In contrast to our usual approach, we create _two pipelines_ in
this notebook. One for feature learning (suffix `_fl`) and one for predicition (suffix `_pr`).
This allows for a fair comparison of runtimes.

In [5]:
pipe_fp_fl = getml.pipeline.Pipeline(
    feature_learners=[feature_learner],
    peripheral=[peripheral],
    population=population,
    tags=["feature learning", "fastprop"],
)

In [6]:
pipe_fp_fl.check(data_train, [data_train])

Checking data model...
OK.


In [7]:
pipe_fp_fl.fit(data_train, [data_train])

Checking data model...
OK.

FastProp: Trying 194 features...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:0.10305



Pipeline(preprocessors=[],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=[],
         tags=['feature learning', 'fastprop'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/occupancy/OoMJb3/0/

In [8]:
fastprop_train = pipe_fp_fl.transform(
    data_train, [data_train], df_name="fastprop_train"
)
fastprop_test = pipe_fp_fl.transform(data_test, [data_test], df_name="fastprop_test")


FastProp: Building features...
[========================================] 100%


FastProp: Building features...
[========================================] 100%



Now we create a dedicated prediction pipeline and provide the fast prop features
(contrained in `fastprop_train` and `fastprop_test`.)

In [9]:
predictor = getml.predictors.XGBoostClassifier()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [10]:
pipe_fp_pr.check(fastprop_train)

pipe_fp_pr.fit(fastprop_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_163' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_165' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_168' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_174' in POPULATION__STAGING_T

Checking data model...
Checking data model...

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:4.057369



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['prediction', 'fastprop'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/occupancy/MFBnbd/0/

In [11]:
pipe_fp_pr.score(fastprop_test)

,date time,set used,target,accuracy,auc,cross entropy
0,2021-05-12 23:29:49,fastprop_train,Occupancy,0.9998,1.,0.0045
1,2021-05-12 23:29:49,fastprop_test,Occupancy,0.9936,0.9986,0.0253


### 2.2 Propositionalization with featuretools

In [12]:
dfs_pandas = {}

for df in getml.project.data_frames:
    dfs_pandas[df.name] = df.to_pandas()
    dfs_pandas[df.name]["id"] = 1

In [13]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(minutes=0),
    memory=pd.Timedelta(minutes=15),
    column_id="id",
    time_stamp="date",
    target="Occupancy",
)

The `FTTimeSeriesBuilder` provides a `fit` method that is designed to be equivilant to
to the `fit` method of the predictorless getML pipeline above.

In [14]:
featuretools_train = ft_builder.fit(dfs_pandas["train"])
featuretools_test = ft_builder.transform(dfs_pandas["test"])

df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=data_train.roles
)
df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=data_train.roles
)

featuretools: Trying features...


/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Time taken: 0h:3m:21.73582



/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [15]:
predictor = getml.predictors.XGBoostClassifier()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['prediction', 'featuretools'],
         share_selected_features=0.5)

In [16]:
pipe_ft_pr.check(df_featuretools_train)

Checking data model...
OK.


In [17]:
pipe_ft_pr.fit(df_featuretools_train)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:0.319378



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['prediction', 'featuretools'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/occupancy/dyy90y/0/

In [18]:
pipe_ft_pr.score(df_featuretools_test)

,date time,set used,target,accuracy,auc,cross entropy
0,2021-05-12 23:37:13,featuretools_train,Occupancy,0.9968,0.9999,0.0118
1,2021-05-12 23:37:13,featuretools_test,Occupancy,0.957,0.9898,0.1304


### 2.3 Propositionalization with tsfresh

In [19]:
tsfresh_builder = TSFreshBuilder(
    num_features=200, memory=15, column_id="id", time_stamp="date", target="Occupancy"
)

tsfresh_train = tsfresh_builder.fit(dfs_pandas["train"])
tsfresh_test = tsfresh_builder.transform(dfs_pandas["test"])

df_tsfresh_train = getml.data.DataFrame.from_pandas(
    tsfresh_train, name="tsfresh_train", roles=data_train.roles
)
df_tsfresh_test = getml.data.DataFrame.from_pandas(
    tsfresh_test, name="tsfresh_test", roles=data_train.roles
)

/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/tsfresh/utilities/dataframe_functions.py:456: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn("Your time stamps are not uniformly sampled, which makes rolling "
Feature Extraction: 100%|██████████| 20/20 [00:16<00:00,  1.22it/s]


Time taken: 0h:0m:47.239761



/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/tsfresh/utilities/dataframe_functions.py:456: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn("Your time stamps are not uniformly sampled, which makes rolling "
Feature Extraction: 100%|██████████| 20/20 [00:21<00:00,  1.08s/it]


In [20]:
pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["predicition", "tsfresh"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['predicition', 'tsfresh'],
         share_selected_features=0.5)

In [21]:
pipe_tsf_pr.check(df_tsfresh_train)

Checking data model...
OK.


In [22]:
pipe_tsf_pr.fit(df_tsfresh_train)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:0.292407



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostClassifier'],
         tags=['predicition', 'tsfresh'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/occupancy/e5o3Cd/0/

In [23]:
pipe_tsf_pr.score(df_tsfresh_test)

,date time,set used,target,accuracy,auc,cross entropy
0,2021-05-12 23:38:54,tsfresh_train,Occupancy,0.9968,0.9999,0.0118
1,2021-05-12 23:38:54,tsfresh_test,Occupancy,0.957,0.9898,0.1304


### 3. Comparison

In [24]:
num_features = dict(
    fastprop=len(pipe_fp_fl.features.filter(lambda feat: "feature" in feat.name)),
    featuretools=featuretools_test.shape[1],
    tsfresh=tsfresh_test[: -len(dfs_pandas["train"].columns)].shape[1],
)

comparison = pd.DataFrame(
    dict(
        runtime=[pipe_fp_fl.runtime, ft_builder.runtime, tsfresh_builder.runtime],
        num_features=num_features.values(),
        runtime_per_feature=[
            f"{pipe_fp_fl.runtime / num_features['fastprop']}",
            f"{ft_builder.runtime / num_features['featuretools']}",
            f"{tsfresh_builder.runtime / num_features['tsfresh']}",
        ],
        auc=[pipe_fp_pr.auc, pipe_ft_pr.auc, pipe_tsf_pr.auc],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

In [25]:
comparison

,runtime,num_features,runtime_per_feature,auc
getML: FastProp,0 days 00:00:00.103050,194,0:00:00.000531,0.998594
featuretools,0 days 00:03:21.735820,92,0:00:02.192781,0.989844
tsfresh,0 days 00:00:47.239761,64,0:00:00.738121,0.989844


## 4. Conclusion

This tutorial demonstrates that relational learning is a powerful tool for time series. We able to outperform the benchmarks for a scientific paper on a simple public domain time series data set using relatively little effort.

If you want to learn more about getML, check out the [official documentation](https://getml.com/product).

# Next Steps

This tutorial went through the basics of applying getML to multirvariate time series and hyperparameter tuning.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.